# Setup original repo

In [1]:
!git clone https://github.com/tensorflow/tpu.git

Cloning into 'tpu'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 9919 (delta 62), reused 64 (delta 35), pack-reused 9798
Receiving objects: 100% (9919/9919), 24.33 MiB | 28.87 MiB/s, done.
Resolving deltas: 100% (7100/7100), done.


In [2]:
!cd tpu/models; pip install -e .

Obtaining file:///content/tpu/models
  Running setup.py develop for cloud-tpu-samples


In [3]:
%cd tpu/models/official/efficientnet/lite

/content/tpu/models/official/efficientnet/lite


# Download original weights

In [5]:
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite0.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite1.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite2.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite3.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite4.tar.gz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 53.5M  100 53.5M    0     0  45.5M      0  0:00:01  0:00:01 --:--:-- 45.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62.4M  100 62.4M    0     0  42.4M      0  0:00:01  0:00:01 --:--:-- 42.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70.1M  100 70.1M    0     0  52.8M      0  0:00:01  0:00:01 --:--:-- 52.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94.4M  100 94.4M    0     0  48.0M      0  0:00:01  0:00:01 --:--:-- 47.9M
  % Total    % Received % Xferd  Average Speed   Tim

# Download example image + labelmap

In [6]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

--2021-04-08 06:41:27--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 142.250.148.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30565 (30K) [text/plain]
Saving to: ‘labels_map.txt’

labels_map.txt      100%[===================>]  29.85K  --.-KB/s    in 0s      

2021-04-08 06:41:27 (62.2 MB/s) - ‘labels_map.txt’ saved [30565/30565]

--2021-04-08 06:41:27--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.153.240, 2620:0:860:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.153.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jp

# Run inference on sample image via Saved Model. 

Repeat these steps for all variants (B0, B1, B2, B3, B4)

Remember to change model_name, ckpt_dir and image_size accordingly/

In [8]:
!rm -rf output
!mkdir output

In [9]:
!python ../export_model.py \
 --model_name=efficientnet-lite0 \
 --ckpt_dir=/content/tpu/models/official/efficientnet/lite/efficientnet-lite0/ \
 --output_saved_model_dir output/temp/ \
 --image_size 224 \
 --output_tflite=output/temp.tflite \
 --quantize=False

2021-04-08 06:44:13.294873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-08 06:44:15.433813: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-08 06:44:15.460489: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-08 06:44:15.535686: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-08 06:44:15.535757: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8b4857891db3): /proc/driver/nvidia/version does not exist
2021-04-08 06:44:15.536340: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
I0408 06:44:15.556524 140170483545984 efficientnet_lite_builder.py:100] glob

In [10]:
import tensorflow as tf
import numpy as np

In [11]:
m = tf.saved_model.load("output/temp")

In [12]:
img = tf.image.decode_jpeg(tf.io.read_file("panda.jpg"))
img = tf.image.resize(img, (224, 224))  # Remember to change this too.
img = tf.expand_dims(img, axis=0)
img.shape

TensorShape([1, 224, 224, 3])

In [13]:
# Persumed preprocessing
img = (img - 127.00) / 128.00

In [14]:
out = m.signatures['serving_default'](img)

In [15]:
print(tf.reduce_max(out['output']))
print(tf.argmax(out['output'], axis=1))

tf.Tensor(0.9809975, shape=(), dtype=float32)
tf.Tensor([388], shape=(1,), dtype=int64)


In [16]:
# Todo: make this:
np.save("/content/b0_lite_output_224.npy", out['output'].numpy())

# Download outputs after running inference will all models:

In [17]:
%cd /content/

/content


In [20]:
!tar czvf original_outs_w_preprocessing.tar.gz b0_lite_output_224.npy b1_lite_output_240.npy b2_lite_output_260.npy b3_lite_output_280.npy b4_lite_output_300.npy

b0_lite_output_224.npy
